# Image Classification Model 

In [1]:
import tensorflow as tf
from keras.applications.resnet import ResNet50, preprocess_input, decode_predictions
import numpy as np

In [2]:
# Setup model
model = ResNet50(weights='imagenet')

102967424/102967424 [==============================] - 1s 0us/step


In [3]:
def predict_resetnet(image_path):
    print("Loading image... %s" % image_path)
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    x = tf.keras.preprocessing.image.img_to_array(img)
    x = np.expand_dims(x, axis=0) 
    x = preprocess_input(x)
    preds = model.predict(x)

    labels = decode_predictions(preds, top=3)[0]
    return labels

In [4]:
predict_resetnet("Aero.jpeg")

Loading image... Aero.jpeg
35363/35363 [==============================] - 0s 0us/step


[('n02690373', 'airliner', 0.98221266),
 ('n04592741', 'wing', 0.010745294),
 ('n04552348', 'warplane', 0.0066661495)]

In [5]:
predict_resetnet("Truck.jpeg")

Loading image... Truck.jpeg
1/1 [==============================] - 0s 30ms/step


[('n04467665', 'trailer_truck', 0.9960282),
 ('n03796401', 'moving_van', 0.0036215454),
 ('n03393912', 'freight_car', 0.00018804525)]

In [6]:
predict_resetnet("Shashank.jpeg")

Loading image... Shashank.jpeg
1/1 [==============================] - 0s 32ms/step


[('n02408429', 'water_buffalo', 0.8243416),
 ('n02403003', 'ox', 0.16601503),
 ('n03868242', 'oxcart', 0.0064453264)]

# Queue

In [7]:
!pip install boto3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.7/134.7 KB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 KB 8.8 MB/s eta 0:00:00


In [8]:
import boto3
import json

In [9]:
aws_access_id = "NULL"
aws_secret_access_key = "NULL"

sqs = boto3.client("sqs", region_name='ap-south-1', aws_access_key_id=aws_access_id, 
                   aws_secret_access_key=aws_secret_access_key)

queue_url = 'https://sqs.ap-south-1.amazonaws.com/249376139068/<QUEUE_NAME>.fifo'

In [24]:
import json

i = 0
while True:
    response = sqs.receive_message(
        QueueUrl=queue_url,
        MaxNumberOfMessages=1,
        WaitTimeSeconds=2
    )

    if 'Messages' in response:
        message = response['Messages'][0]
        print( predict_resetnet( json.loads(message["Body"])["path"] ) )
        print(f"new messsage received: {message['Body']}")
        i += 1
    # else:
        # print("No Message Received")
        
    if i >= 5:
        break

Loading image... Aero.jpeg
1/1 [==============================] - 0s 28ms/step
[('n02690373', 'airliner', 0.98221266), ('n04592741', 'wing', 0.010745294), ('n04552348', 'warplane', 0.0066661495)]
new messsage received: {"path":"Aero.jpeg"}
Loading image... Truck.jpeg
1/1 [==============================] - 0s 27ms/step
[('n04467665', 'trailer_truck', 0.9960282), ('n03796401', 'moving_van', 0.0036215454), ('n03393912', 'freight_car', 0.00018804525)]
new messsage received: {"path":"Truck.jpeg"}
Loading image... Shashank.jpeg
1/1 [==============================] - 0s 34ms/step
[('n02408429', 'water_buffalo', 0.8243416), ('n02403003', 'ox', 0.16601503), ('n03868242', 'oxcart', 0.0064453264)]
new messsage received: {"path":"Shashank.jpeg"}
Loading image... Aero.jpeg
1/1 [==============================] - 0s 26ms/step
[('n02690373', 'airliner', 0.98221266), ('n04592741', 'wing', 0.010745294), ('n04552348', 'warplane', 0.0066661495)]
new messsage received: {"path":"Aero.jpeg"}
Loading image...

In [ ]:
    print("done")

done


# S3 operations

In [11]:
bucket = "img-service-bucket"

## Upload

In [12]:
s3 = boto3.client("s3", region_name='ap-south-1', aws_access_key_id=aws_access_id, 
                   aws_secret_access_key=aws_secret_access_key)

In [15]:
file_name = "Aero.jpeg"

In [16]:
s3.upload_file(file_name, bucket, "uploads/" + file_name)

## List

In [17]:
s3_resource = boto3.resource("s3", region_name='ap-south-1', aws_access_key_id=aws_access_id, 
                   aws_secret_access_key=aws_secret_access_key)
s3_bucket = s3_resource.Bucket(bucket)

In [18]:
all_files = s3_bucket.objects.all()

In [19]:
for file in all_files:
  print(file.key)

uploads/
uploads/Aero.jpeg
uploads/Shashank.jpeg
uploads/Truck.jpeg


## Download

In [20]:
s3_object = s3_resource.Object(bucket, 'uploads/Aero.jpeg')
s3_object.download_file('p1_tmp.jpg')